# Tentativa #1

In [18]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.ml.feature import VectorAssembler, StringIndexer, StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier  # or any other algorithm
from pyspark.ml.evaluation import BinaryClassificationEvaluator  # or MulticlassClassificationEvaluator
from pyspark.sql.functions import coalesce, lit

## 0. Initialize Spark Session

In [19]:
spark = SparkSession.builder \
    .appName("BigDataMLProject") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .getOrCreate()

## 1. Load Data locally

In [20]:
DATA_PATH = "./mimic-db-short"

items = spark.read.csv(f"{DATA_PATH}/D_ITEMS.csv", header=True, inferSchema=True)
procedures = spark.read.csv(f"{DATA_PATH}/PROCEDURES_ICD.csv", header=True, inferSchema=True)

chartevents = spark.read.csv(f"{DATA_PATH}/CHARTEVENTS.csv", header=True, inferSchema=True)
patients = spark.read.csv(f"{DATA_PATH}/PATIENTS.csv", header=True, inferSchema=True)
admissions = spark.read.csv(f"{DATA_PATH}/ADMISSIONS.csv", header=True, inferSchema=True)
diagnoses = spark.read.csv(f"{DATA_PATH}/DIAGNOSES_ICD.csv", header=True, inferSchema=True)
icustays = spark.read.csv(f"{DATA_PATH}/ICUSTAYS.csv", header=True, inferSchema=True)
inputevents = spark.read.csv(f"{DATA_PATH}/INPUTEVENTS_MV.csv", header=True, inferSchema=True)
labevents = spark.read.csv(f"{DATA_PATH}/LABEVENTS.csv", header=True, inferSchema=True)

print("Data Loaded!")


Data Loaded!


## 2. Data Preprocessing

The tables were chose based on its properties regarding a persons' illness, bodily atributes or gravity of the situation. The tables were:
* ChartEVents
* Admissions
* Patients
* Diagnoses
* ICUStays
* InputEvents_MV
* Procedures

### A. Feature Engineering

The majoriry of the atributes are not useful to predict the duration of a ICU stay, therefore, for a initial analysis, we will choose the ones we consider relevant.

First of all, we create temporary views of all tables into a SQL-like table.

In [21]:
chartevents.createOrReplaceTempView("chartevents")
patients.createOrReplaceTempView("patients")
admissions.createOrReplaceTempView("admissions")
diagnoses.createOrReplaceTempView("diagnoses")
icustays.createOrReplaceTempView("icustays")
inputevents.createOrReplaceTempView("inputevents")
labevents.createOrReplaceTempView("labevents")
procedures.createOrReplaceTempView("procedures")

SO QUEREMOS PESSOAS QUE TENHAM ICUSTAY_ID PQ: given a person admitted int he icu i have to predict for how long they are gonna be there, therefore in pre processing i will ignore the data from people that were never in the icu. The most efficient approach is to create temporary views that pre-filter the data before joining tables.

In [22]:
spark.sql("CREATE OR REPLACE TEMPORARY VIEW icu_filtered AS SELECT * FROM icustays WHERE icustay_id IS NOT NULL")
spark.sql("CREATE OR REPLACE TEMPORARY VIEW chart_filtered AS SELECT * FROM chartevents WHERE icustay_id IS NOT NULL")
spark.sql("CREATE OR REPLACE TEMPORARY VIEW input_filtered AS SELECT * FROM inputevents WHERE icustay_id IS NOT NULL")

DataFrame[]

To get procedures done to icu patients

In [23]:
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW proc_filtered AS
SELECT 
    p.icd9_code,
    icu.icustay_id
FROM procedures p
JOIN icustays icu ON p.hadm_id = icu.hadm_id
""")

DataFrame[]

To handle labevents time to be only during the icustay: get only one row by labevent, the first valid (not null result) the person did when enterying the icu. To predcit the legth someone will stay, when they first enter, we will only have the first test, so we will train the machine learning using this first value.

In [24]:
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW lab_filtered AS
SELECT 
    lab.subject_id,
    lab.hadm_id,
    lab.itemid,
    lab.valuenum,
    lab.valueuom,
    icu.icustay_id 
FROM labevents lab
JOIN admissions adm ON lab.hadm_id = adm.hadm_id
JOIN icustays icu ON adm.hadm_id = icu.hadm_id
WHERE 
    lab.charttime BETWEEN icu.intime AND icu.outtime
    AND lab.hadm_id IS NOT NULL
""")

spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW lab_filtered2 AS
SELECT 
    subject_id,
    hadm_id,
    itemid,
    FIRST_VALUE(valuenum) IGNORE NULLS OVER (
        PARTITION BY icustay_id, itemid 
        ORDER BY charttime
    ) AS valuenum,
    icustay_id
FROM lab_filtered
""")


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `charttime` cannot be resolved. Did you mean one of the following? [`hadm_id`, `itemid`, `valuenum`, `valueuom`, `icustay_id`].; line 9 pos 17;
'CreateViewCommand `lab_filtered2`, SELECT 
    subject_id,
    hadm_id,
    itemid,
    FIRST_VALUE(valuenum) IGNORE NULLS OVER (
        PARTITION BY icustay_id, itemid 
        ORDER BY charttime
    ) AS valuenum,
    icustay_id
FROM lab_filtered, false, true, LocalTempView, false
+- 'Project [subject_id#1302, hadm_id#1303, itemid#1304, first(valuenum#1305, true) windowspecdefinition(icustay_id#1307, itemid#1304, 'charttime ASC NULLS FIRST, specifiedwindowframe(RangeFrame, unboundedpreceding$(), currentrow$())) AS valuenum#1301, icustay_id#1307]
   +- SubqueryAlias lab_filtered
      +- View (`lab_filtered`, [subject_id#1302,hadm_id#1303,itemid#1304,valuenum#1305,valueuom#1306,icustay_id#1307])
         +- Project [cast(subject_id#1218 as int) AS

Then, we create a table, using a SQL query, with the relevant features for predicting the icu stay duration, and its own column (LOS from ICUStays).

In [ ]:
df = spark.sql("""
    SELECT 
        icu.icustay_id,
        adm.admission_type,
        adm.admission_location,
        chart.itemid AS chart_itemid,
        chart.valuenum AS chart_value,
        chart.error,
        lab.itemid AS lab_itemid,
        lab.valuenum AS lab_value,
        diag.seq_num,
        diag.icd9_code as diagnose_code,
        input.itemid AS input_itemid,
        input.amount,
        input.rate,
        input.patientweight,
        proc.icd9_code as procedure_code,
        icu.first_careunit,
        icu.LOS
    FROM icu_filtered icu
    JOIN admissions adm ON icu.hadm_id = adm.hadm_id
    LEFT JOIN chart_filtered chart ON icu.icustay_id = chart.icustay_id
    LEFT JOIN lab_filtered2 lab ON icu.icustay_id = lab.icustay_id
    LEFT JOIN diagnoses diag ON icu.hadm_id = diag.hadm_id
    LEFT JOIN input_filtered input ON icu.icustay_id = input.icustay_id
    LEFT JOIN proc_filtered proc ON proc.icustay_id=icu.icustay_id
""")
df.show()

+----------+--------------+--------------------+------------+-----------+-----+----------+---------+-------+-------------+------------+------+----+-------------+--------------+--------------+-------+
|icustay_id|admission_type|  admission_location|chart_itemid|chart_value|error|lab_itemid|lab_value|seq_num|diagnose_code|input_itemid|amount|rate|patientweight|procedure_code|first_careunit|    LOS|
+----------+--------------+--------------------+------------+-----------+-----+----------+---------+-------+-------------+------------+------+----+-------------+--------------+--------------+-------+
|    285977|        URGENT|TRANSFER FROM HOS...|        NULL|       NULL| NULL|      NULL|     NULL|   NULL|         NULL|        NULL|  NULL|NULL|         NULL|          NULL|          CSRU| 0.8181|
|    279205|     EMERGENCY|EMERGENCY ROOM ADMIT|        NULL|       NULL| NULL|      NULL|     NULL|   NULL|         NULL|        NULL|  NULL|NULL|         NULL|          NULL|         TSICU| 6.6602|


merdas para justificar no relatorio:

Juntei as tabelas icu e admissions por admissao ao hospital e nao por pessoa, pq pode haver mais doq uma admissao por pessoa. Fiz outter join pelo msm motivo, mas dps tenho de testar isso com os dados todos e nao so com as tabelas parciais

na chartevents, juntei por icu stay, Each row associated with one ITEMID (e.g. 212) corresponds to an instantiation of the same measurement (e.g. heart rate) entao nao vale a pena por a coluna VALUEUOM is the unit of measurement, pq cada teste ja e feito numa unidade de medida, n vale a pena tar a dar mais do msm a ml. Vou por o erro pq se houve erro ent conta menos para a ml mas ig que e melhor q nada??? problema para a ml e nao para nos.

aqui cada chartevent, ou seja, analie/teste/raiox etc corresponde as uma linha, mais tarde temos de resolver este problema pq o objetivo e prever o tempo de internacao POR ICUSTAY_ID, ent deviamos ter apenas uma linha por admissao//icustay

na tabela dos diagnosticos SEQ_NUM provides the order in which the ICD diagnoses relate to the patient, o quao importante a doenca e no caso da pessoa, e ICD9_CODE contains the actual code corresponding to the diagnosis, ou seja a doenca id, por isso para dar join dou por paciente ou por admissao? aceito opinioes, por agr pus admissoes pq e oq pus no resto

input events dei join pela icustay ja que so queremos analisar pacientes que ja tenham estado. Each row associated with one ITEMID which corresponds to an instantiation of the same measurement (e.g. norepinephrine) AMOUNT - the amount of a drug or substance administered to the patient (ignorando se esta em ml dl ou l pelo motivo referido acima)


When predicting ICU length of stay (LOS), incorporating lab results can significantly improve your model's performance, but you're right that joining requires careful handling since LABEVENTS doesn't directly contain ICU stay IDs. juntar pelo ham id ou subject e dps escolher com base no tempo


the id is the icustays

justificar o left join:
For your use case, keep the LEFT JOINs but understand why:

    LEFT JOIN (icu → others) is correct because:

        You want all ICU stays (base table)

        You want to keep ICU stays even if they're missing some diagnoses/procedures

    Don't use FULL OUTER JOIN because:

        It would include diagnoses/procedures for non-ICU patients (if any exist)

        Could create NULL ICU stay records which you don't want

    Your current approach is good for:

        One row per combination of ICU stay + diagnosis + procedure

        Preserving all relationships

Example of What You'll Get

For a patient with:

    1 ICU stay

    4 diagnoses

    5 procedures

    10 lab results

Your query will produce 4 × 5 × 10 = 200 rows for this patient (all combinations).


FALTA:
altura / bmi do paciente
genero do paciente

Attribute/column documentation for the table created:

 1. Patient Identification
- **icustay_id**: Unique ICU stay identifier (primary key for ICU stays)

 2. Admission Information
- **admission_type**: Type of admission (ELECTIVE, EMERGENCY, URGENT, etc.)
- **admission_location**: Source of admission (TRANSFER FROM HOSPITAL, CLINIC REFERRAL, etc.)
- **first_careunit**: Initial ICU care unit (MICU, SICU, CSRU, etc.)
- **LOS**: Length of stay in ICU (in hours or days)

 3. Clinical Measurements
- **chart_itemid**: Identifier for charted measurement (foreign key to D_ITEMS)
- **chart_value**: Numeric value of the clinical measurement
- **chart_error**: Error flag for the measurement (if exists)
- **lab_itemid**: Identifier for laboratory test (foreign key to D_ITEMS)
- **lab_value**: Result value of the laboratory test

 4. Diagnostic Information
- **diagnose_code**: ICD-9 diagnosis code
- **seq_num**: Priority/sequence number of the diagnosis (1=primary)

 5. Treatment Information
- **input_itemid**: Identifier for input event (medications/fluids)
- **input_amount**: Quantity administered
- **input_rate**: Rate of administration
- **input_patientweight**: Patient weight at time of input (if recorded)

 6. Procedural Information
- **procedure_code**: ICD-9 procedure code performed during stay

transform your multiple rows per ICU stay into a single row format suitable for ML modeling while preserving both continuous values and procedure/diagnosis information

### B. Get one row by icu stay ID - GHOST CODE 

Handle LabEvents:

In [ ]:
# Register the DataFrame as a temporary view
df.createOrReplaceTempView("original_data")

# Step 1 & 2: Pivot in a single SQL operation
pivot_query = """
WITH lab_data AS (
    SELECT 
        icustay_id,
        lab_itemid,
        FIRST(lab_value) OVER (PARTITION BY icustay_id, lab_itemid ORDER BY lab_value) AS lab_value
    FROM original_data
    WHERE lab_itemid IS NOT NULL
    GROUP BY icustay_id, lab_itemid, lab_value
)
SELECT 
    icustay_id,
    {pivot_columns}
FROM lab_data
PIVOT (
    FIRST(lab_value)
    FOR lab_itemid IN ({itemid_list})
)
"""

# Get distinct lab_itemids (for large datasets, consider sampling first)
itemids = [str(row['lab_itemid']) for row in df.filter("lab_itemid IS NOT NULL").select("lab_itemid").distinct().collect()]

# Generate the pivot columns and IN clause
pivot_columns = ", ".join(itemids)
itemid_list = ", ".join([f"'{id}'" for id in itemids])

# Execute the pivot
lab_pivoted = spark.sql(pivot_query.format(
    pivot_columns=pivot_columns,
    itemid_list=itemid_list
))

# Step 3: Fill nulls (now in SQL)
fill_nulls_query = """
SELECT
    icustay_id,
    {coalesce_expressions}
FROM lab_pivoted
"""

coalesce_exprs = [f"COALESCE({col}, -1) AS {col}" for col in itemids]
lab_pivoted_filled = spark.sql(fill_nulls_query.format(
    coalesce_expressions=",\n    ".join(coalesce_exprs)
))

# Step 4 & 5: Final join in a single SQL operation
final_query = """
SELECT 
    base.*,
    {lab_columns}
FROM (
    SELECT 
        icustay_id,
        {base_columns}
    FROM original_data
    GROUP BY icustay_id, {base_columns}
) base
LEFT JOIN lab_pivoted_filled lab ON base.icustay_id = lab.icustay_id
"""

# Get all columns except lab columns
base_cols = [col for col in df.columns if col not in ["lab_itemid", "lab_value"]]
lab_cols = ", ".join(itemids)

df_final = spark.sql(final_query.format(
    lab_columns=lab_cols,
    base_columns=", ".join(base_cols),
    base_columns_list=", ".join(base_cols)
))


Handle InputEvents:

In [ ]:
# First, register the DataFrame as a temporary view
df.createOrReplaceTempView("combined_data")

# Step 1: Pivot input events in SQL (more efficient for big data)
input_pivot_sql = """
SELECT 
    icustay_id,
    {pivot_expressions}
FROM (
    SELECT 
        icustay_id,
        input_itemid,
        amount,
        rate,
        patientweight,
        ROW_NUMBER() OVER (PARTITION BY icustay_id, input_itemid ORDER BY amount DESC) as rn
    FROM combined_data
    WHERE input_itemid IS NOT NULL
) 
WHERE rn = 1  -- Get one record per icustay_id and input_itemid
GROUP BY icustay_id
"""

# Dynamically generate pivot expressions for all itemids
# First get distinct itemids (for big data, sample first if too many)
itemids = spark.sql("SELECT DISTINCT input_itemid FROM combined_data WHERE input_itemid IS NOT NULL").collect()
itemids = [str(row['input_itemid']) for row in itemids]

pivot_exprs = []
for itemid in itemids:
    pivot_exprs.append(f"MAX(CASE WHEN input_itemid = {itemid} THEN amount END) AS {itemid}_amount")
    pivot_exprs.append(f"MAX(CASE WHEN input_itemid = {itemid} THEN rate END) AS {itemid}_rate")

pivot_sql = input_pivot_sql.format(pivot_expressions=",\n    ".join(pivot_exprs))

# Execute the pivot
input_pivoted = spark.sql(pivot_sql)

# Step 2: Get the latest patientweight (using SQL)
patientweight_sql = """
SELECT 
    icustay_id,
    LAST(patientweight, true) as patientweight  -- true ignores nulls
FROM combined_data
WHERE patientweight IS NOT NULL
GROUP BY icustay_id
"""
patientweight_df = spark.sql(patientweight_sql)

# Step 3: Join the pivoted data with patientweight
input_final_sql = """
SELECT 
    p.*,
    w.patientweight
FROM input_pivoted p
LEFT JOIN patientweight_df w ON p.icustay_id = w.icustay_id
"""
input_final = spark.sql(input_final_sql)

# Step 4: Create final DataFrame by joining with original data (minus input columns)
final_sql = """
SELECT 
    orig.*,
    {input_columns},
    inp.patientweight
FROM (
    SELECT DISTINCT 
        icustay_id,
        admission_type,
        admission_location,
        chart_itemid,
        chart_value,
        error,
        lab_itemid,
        lab_value,
        seq_num,
        diagnose_code,
        procedure_code,
        first_careunit,
        LOS
    FROM combined_data
) orig
LEFT JOIN input_final inp ON orig.icustay_id = inp.icustay_id
"""

# Generate column list for input columns
input_columns = [f"inp.{itemid}_amount, inp.{itemid}_rate" for itemid in itemids]
input_columns = ",\n    ".join(input_columns)

final_sql = final_sql.format(input_columns=input_columns)

# Execute final query
df_final = spark.sql(final_sql)

# Fill nulls with -1 for input columns
input_cols = [f"{itemid}_amount" for itemid in itemids] + [f"{itemid}_rate" for itemid in itemids]
df_final = df_final.fillna(-1, subset=input_cols)

Handle Procedures:

Ajustar o tamanho dos top procedures para tmb n ter uma tabela com 50000000 colunas

In [ ]:
# 1. First get top N most frequent procedures
top_procedures = df_final.filter("procedure_code IS NOT NULL") \
                         .groupBy("procedure_code") \
                         .count() \
                         .orderBy("count", ascending=False) \
                         .limit(100) \
                         .collect()
top_procedure_codes = [row['procedure_code'] for row in top_procedures]

# 2. Create one-hot encoded columns using CASE WHEN
select_exprs = [
    f"MAX(CASE WHEN procedure_code = '{code}' THEN 1 ELSE 0 END) AS procedure_{code}"
    for code in top_procedure_codes
]

procedure_encoded = df_final.groupBy("icustay_id") \
                           .agg(*select_exprs)

# 3. Join back to main DataFrame
df_final_with_procedures = df_final.join(procedure_encoded, on="icustay_id", how="left")

Handle Diagnostics:

In [ ]:
# Step 1: Calculate importance score (1/seq_num) and get the most important record
window_spec = Window.partitionBy("icustay_id", "diagnose_code").orderBy("seq_num")

diagnoses = (df_final
    .filter(F.col("diagnose_code").isNotNull())
    .withColumn("importance", F.lit(1)/F.col("seq_num"))  # Calculate 1/seq_num
    .withColumn("rn", F.row_number().over(window_spec))
    .filter(F.col("rn") == 1)  # Take only the first occurrence
    .select("icustay_id", "diagnose_code", "importance")
)

# Step 2: Pivot with importance scores
diag_pivoted = (diagnoses
    .groupBy("icustay_id")
    .pivot("diagnose_code")
    .agg(F.first("importance"))  # Use the importance score instead of seq_num
    .fillna(0)  # 0 indicates diagnosis not present (since 1/seq_num will always be > 0)
)

# Rename columns to add 'diag_' prefix
for col in diag_pivoted.columns:
    if col != "icustay_id":
        diag_pivoted = diag_pivoted.withColumnRenamed(col, f"diag_{col}")

# Step 3: Join back to main DataFrame
df_final_with_diag = df_final.join(diag_pivoted, on="icustay_id", how="left")

In [ ]:
df_ready = df_final_with_diag
df_ready.show()

### C. Every entry has to be numeric

See data types of each column

In [ ]:
df_ready.schema

### D. Scale features

## 3. Target variable preparation (if categorical)

In [ ]:
label_indexer = StringIndexer(inputCol=target_col, outputCol="label")

## 4. Create Pipeline

In [ ]:
pipeline_stages = indexers + [assembler, scaler, label_indexer]
preprocessing_pipeline = Pipeline(stages=pipeline_stages)

In [ ]:
processed_data = preprocessing_pipeline.fit(df).transform(df)

## 5. Train-Test Split

In [ ]:
train_data, test_data = processed_data.randomSplit([0.7, 0.3], seed=42)
print(f"Training count: {train_data.count()}")
print(f"Test count: {test_data.count()}")

## 6. Model Training

In [ ]:
rf = RandomForestClassifier(
    featuresCol="features",
    labelCol="label",
    numTrees=100,
    maxDepth=10,
    seed=42
)

# Train model
model = rf.fit(train_data)

## 7. Predictions

In [ ]:
predictions = model.transform(test_data)

## 8. Evaluation

In [ ]:
evaluator = BinaryClassificationEvaluator(labelCol="label")
auc = evaluator.evaluate(predictions)
print(f"Test AUC = {auc}")

In [ ]:
# For multiclass classification
# evaluator = MulticlassClassificationEvaluator(labelCol="label", metricName="accuracy")
# accuracy = evaluator.evaluate(predictions)
# print(f"Test Accuracy = {accuracy}")

## 9. Close Session

In [ ]:
spark.stop()